In [1]:
import pandas as pd
#Dict approach
import glob
import os
import sys
from pathlib import Path
#from typing import Any, Optional
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
#import pytorch_lightning as pl
import h5py
import timeit
import pysam
from pysam.libcalignmentfile import AlignmentFile

#Load genome loader
sys.path.append('/iblm/netapp/home/jjaureguy/genome_loader/genome-loader')

sample_path = os.listdir('/iblm/netapp/data3/jjaureguy/PRJEB18997/10_genos/10_genos_fastq')
genos_path = os.listdir('/iblm/netapp/data3/jjaureguy/PRJEB18997/10_genos/19_genos_fastq')
test_fasta= '/iblm/netapp/data4/jjaureguy/ref_genome/hg38_ref/genome.fa'
out_dir = "/iblm/netapp/data4/jjaureguy/out_dir"
treatments_path = os.listdir('/iblm/netapp/data3/jjaureguy/PRJEB18997/10_genos/10_genos_fastq/HPSI0114i-eipl_1/out/treatments')
total_path = sample_path + genos_path

import glob

In [2]:
import genome_loader.write_h5
import genome_loader.encode_data
import genome_loader.get_data
import genome_loader.get_encoded
import genome_loader.load_data
import genome_loader.load_h5
from genome_loader.write_h5 import write_encoded_genome
from genome_loader.write_h5 import write_frag_depth
from genome_loader.get_data import get_frag_depth

In [3]:
# Check chromosome naming in list
unified_bed_df = pd.read_csv('/iblm/netapp/data3/jjaureguy/PRJEB18997/10_genos/text_files/unified_peak_set/unified_peak.txt',sep='\t', header=None, names=['chrom', 'start', 'end'])

unique = unified_bed_df['chrom'].unique()
# generate a list of chromosomes 1-22 without X and Y and mitochondrial 
chr_list =  [ 'chr{}'.format(x) for x in list(range(1,23)) ]


In [4]:
chr_list

['chr1',
 'chr2',
 'chr3',
 'chr4',
 'chr5',
 'chr6',
 'chr7',
 'chr8',
 'chr9',
 'chr10',
 'chr11',
 'chr12',
 'chr13',
 'chr14',
 'chr15',
 'chr16',
 'chr17',
 'chr18',
 'chr19',
 'chr20',
 'chr21',
 'chr22']

In [5]:
bam_paths = []
peak_paths = []
treatment_paths = []

column_names = [
    #'patient id', 
                'treatments','bam paths', 'peak paths']

d = {}

#filenames = [f for f in filenames if (f.startswith("ERR") and f.lower().endswith("1"))]
for i in (sample_path):
    #print([i][0]
    path = f"/iblm/netapp/data3/jjaureguy/PRJEB18997/10_genos/10_genos_fastq/{i}/out/treatments/*/merged_qc.bam"
    peak_path = f"/iblm/netapp/data3/jjaureguy/PRJEB18997/10_genos/10_genos_fastq/{i}/out/treatments/merged.bed"
    for filename in glob.glob(path):
        bam_paths.append(filename)
    for filename in glob.glob(peak_path):
        peak_paths.append(filename)
    for filename in (treatments_path):
        treatment_paths.append(filename)
    d[i] = pd.DataFrame(bam_paths)
    d[i].columns = ['bam paths']
    #d[i]['patient id'] = i
    d[i]['peak paths'] = pd.Series(peak_paths)
    d[i]['peak paths']= d[i]['peak paths'].fillna(method='ffill')
    d[i]['treatments']= pd.Series(treatment_paths)
    d[i]['treatments'] = d[i]['treatments'].str.replace('_mb', '')
    d[i] = d[i].reindex(columns=column_names)
    bam_paths.clear()
    peak_paths.clear()
    treatment_paths.clear()


In [6]:
import glob
bam_paths = []
peak_paths = []
treatment_paths = []

column_names = [
    #'patient id', 
                'treatments','bam paths', 'peak paths']

d_add = {}

#filenames = [f for f in filenames if (f.startswith("ERR") and f.lower().endswith("1"))]
for i in (genos_path):
    #print([i][0]
    path = f"/iblm/netapp/data3/jjaureguy/PRJEB18997/10_genos/19_genos_fastq/{i}/out/treatments/*/merged_qc.bam"
    peak_path = f"/iblm/netapp/data3/jjaureguy/PRJEB18997/10_genos/19_genos_fastq/{i}/out/treatments/merged.bed"
    for filename in glob.glob(path):
        bam_paths.append(filename)
    for filename in glob.glob(peak_path):
        peak_paths.append(filename)
    for filename in (treatments_path):
        treatment_paths.append(filename)
    d_add[i] = pd.DataFrame(bam_paths)
    d_add[i].columns = ['bam paths']
    #d[i]['patient id'] = i
    d_add[i]['peak paths'] = pd.Series(peak_paths)
    d_add[i]['peak paths']= d_add[i]['peak paths'].fillna(method='ffill')
    d_add[i]['treatments']= pd.Series(treatment_paths)
    d_add[i]['treatments'] = d_add[i]['treatments'].str.replace('_mb', '')
    d_add[i] = d_add[i].reindex(columns=column_names)
    bam_paths.clear()
    peak_paths.clear()
    treatment_paths.clear()

df = pd.concat(d).reset_index().drop(columns='level_1').rename(columns={'level_0': 'patient_id'})
df_19 = pd.concat(d_add).reset_index().drop(columns='level_1').rename(columns={'level_0': 'patient_id'})

frames = [df, df_19]

result = pd.concat(frames)



In [7]:
result

,patient_id,treatments,bam paths,peak paths
0,HPSI0114i-eipl_1,diff,/iblm/netapp/data3/jjaureguy/PRJEB18997/10_gen...,/iblm/netapp/data3/jjaureguy/PRJEB18997/10_gen...
1,HPSI0114i-eipl_1,int_gamma,/iblm/netapp/data3/jjaureguy/PRJEB18997/10_gen...,/iblm/netapp/data3/jjaureguy/PRJEB18997/10_gen...
2,HPSI0114i-eipl_1,salm_int_gamma,/iblm/netapp/data3/jjaureguy/PRJEB18997/10_gen...,/iblm/netapp/data3/jjaureguy/PRJEB18997/10_gen...
3,HPSI0114i-eipl_1,salm,/iblm/netapp/data3/jjaureguy/PRJEB18997/10_gen...,/iblm/netapp/data3/jjaureguy/PRJEB18997/10_gen...
4,HPSI0114i-joxm_1,diff,/iblm/netapp/data3/jjaureguy/PRJEB18997/10_gen...,/iblm/netapp/data3/jjaureguy/PRJEB18997/10_gen...
...,...,...,...,...
61,HPSI0214i-eiwy_1,salm,/iblm/netapp/data3/jjaureguy/PRJEB18997/10_gen...,/iblm/netapp/data3/jjaureguy/PRJEB18997/10_gen...
62,HPSI1113i-podx_1,diff,/iblm/netapp/data3/jjaureguy/PRJEB18997/10_gen...,/iblm/netapp/data3/jjaureguy/PRJEB18997/10_gen...
63,HPSI1113i-podx_1,int_gamma,/iblm/netapp/data3/jjaureguy/PRJEB18997/10_gen...,/iblm/netapp/data3/jjaureguy/PRJEB18997/10_gen...
64,HPSI1113i-podx_1,salm_int_gamma,/iblm/netapp/data3/jjaureguy/PRJEB18997/10_gen...,/iblm/netapp/data3/jjaureguy/PRJEB18997/10_gen...


In [8]:
from tqdm.notebook import tqdm

cur_offset = 0
bed_dfs = {}
for (patient_id, bam_path, peak_file), pdf in tqdm(result.groupby(['patient_id','bam paths', 'peak paths'])):
    
    bed_df = pd.read_csv(peak_file, delimiter='\t', 
                         header=None, names=['chrom', 'start', 'end'], 
                         dtype={'chrom':'category', 'start':int, 'end': int})
    
    bed_df['bam paths'] = bam_path
    bed_df['peak paths'] = peak_path
    bed_dfs[patient_id] = bed_df
dataset_df = pd.concat(bed_dfs, names=['patient_id']).reset_index()

del dataset_df['level_1']

#prev = dataset_df['bam paths'].unique()
x = result['bam paths'].unique()

#Add aarons new functions write_read_depth for frags(Tn5 cut sites)


  0%|          | 0/104 [00:00<?, ?it/s]

In [9]:
x

array(['/iblm/netapp/data3/jjaureguy/PRJEB18997/10_genos/10_genos_fastq/HPSI0114i-eipl_1/out/treatments/diff_mb/merged_qc.bam',
       '/iblm/netapp/data3/jjaureguy/PRJEB18997/10_genos/10_genos_fastq/HPSI0114i-eipl_1/out/treatments/int_gamma_mb/merged_qc.bam',
       '/iblm/netapp/data3/jjaureguy/PRJEB18997/10_genos/10_genos_fastq/HPSI0114i-eipl_1/out/treatments/salm_int_gamma_mb/merged_qc.bam',
       '/iblm/netapp/data3/jjaureguy/PRJEB18997/10_genos/10_genos_fastq/HPSI0114i-eipl_1/out/treatments/salm_mb/merged_qc.bam',
       '/iblm/netapp/data3/jjaureguy/PRJEB18997/10_genos/10_genos_fastq/HPSI0114i-joxm_1/out/treatments/diff_mb/merged_qc.bam',
       '/iblm/netapp/data3/jjaureguy/PRJEB18997/10_genos/10_genos_fastq/HPSI0114i-joxm_1/out/treatments/int_gamma_mb/merged_qc.bam',
       '/iblm/netapp/data3/jjaureguy/PRJEB18997/10_genos/10_genos_fastq/HPSI0114i-oevr_3/out/treatments/diff_mb/merged_qc.bam',
       '/iblm/netapp/data3/jjaureguy/PRJEB18997/10_genos/10_genos_fastq/HPSI0114i-oe

In [ ]:
for fn in tqdm(x):
    fnp = Path(fn)
    outdir = fnp.parent
    write_frag_depth(str(fnp), str(outdir),  chrom_list=chr_list)

  0%|          | 0/104 [00:00<?, ?it/s]

Counted chr1 fragments in 14.71 seconds!
Counted chr2 fragments in 14.65 seconds!
Counted chr3 fragments in 12.47 seconds!
Counted chr4 fragments in 10.55 seconds!
Counted chr5 fragments in 10.90 seconds!
Counted chr6 fragments in 9.92 seconds!
Counted chr7 fragments in 9.52 seconds!
Counted chr8 fragments in 8.35 seconds!
Counted chr9 fragments in 6.74 seconds!
Counted chr10 fragments in 8.08 seconds!
Counted chr11 fragments in 8.24 seconds!
Counted chr12 fragments in 7.71 seconds!
Counted chr13 fragments in 5.00 seconds!
Counted chr14 fragments in 5.08 seconds!
Counted chr15 fragments in 4.53 seconds!
Counted chr16 fragments in 4.81 seconds!
Counted chr17 fragments in 4.11 seconds!
Counted chr18 fragments in 4.11 seconds!
Counted chr19 fragments in 3.38 seconds!
Counted chr20 fragments in 3.46 seconds!
Counted chr21 fragments in 1.61 seconds!
Counted chr22 fragments in 1.51 seconds!
Processed fragment data in 159.47 seconds!
Finished writing in 18.73 seconds!
Frag-Depths written to /

In [ ]:
print('done')

In [ ]:
def get_frag_depth(in_bam, chrom_list=chr_list, chrom_lens=None):
    start_time = timeit.default_timer()
    
    depth_dict = {}
    
    with AlignmentFile(in_bam, "r") as bam:

        # Check valid chrom list and lengths
        if not chrom_list or not chrom_lens:
            if not chrom_list:
                chrom_list = list(bam.header.references)
            
            chrom_lens = {chrom:bam.header.get_reference_length(chrom)
                          for chrom in chrom_list}

        elif not isinstance(chrom_lens, dict):
            chrom_lens = {chrom:clen for chrom, clen in zip(chrom_list, chrom_lens)}
        
        # Parse through chroms
        for chrom in chrom_list:
            start_chrom = timeit.default_timer()

            curr_len = chrom_lens[chrom]
            out_array = np.zeros(curr_len, dtype=np.uint16)
            
            read_cache = {}

            for read in bam.fetch(chrom):

                if not read.is_proper_pair or read.is_secondary:
                    continue

                if read.query_name not in read_cache:
                    read_cache[read.query_name] = read
                    continue

                # Forward and Reverse w/o r1 and r2
                if read.is_reverse:
                    forward_read = read_cache.pop(read.query_name)
                    reverse_read = read
                else:
                    forward_read = read
                    reverse_read = read_cache.pop(read.query_name)

                # Shift read
                forward_start = forward_read.reference_start + 4
                reverse_end = reverse_read.reference_end - 5 # 0 based, 1 past aligned

                # Add cut sites to out_array
                out_array[[forward_start, (reverse_end-1)]] += 1
                
                # Add midpoint to out_array
                # out_array[int((forward_start+(reverse_end-1))/2)] += 1
                
                # Add range to out array
                #out_array[forward_start:reverse_end] += 1    
            
            out_array[np.where(out_array > 255)] = 255
            out_array = out_array.astype(np.uint8)
            
            depth_dict[chrom] = out_array
            
            print(f"Counted {chrom} fragments in {timeit.default_timer() - start_chrom} seconds!")

    print(f"Processed fragment data in {timeit.default_timer() - start_time} seconds!")
    return depth_dict

def write_frag_depth(in_bam, out_dir, h5_name=None, chrom_list=None, chrom_lens=None):

    if h5_name:
        out_h5 = str(Path(out_dir) / h5_name)
    else:
        out_h5 = str(Path(out_dir) / "frag_depths.h5")
    
    # Get data using read depth function
    depth_dict = get_frag_depth(in_bam, chrom_list=chrom_list, chrom_lens=chrom_lens)
    
    start_write = timeit.default_timer()
    with h5py.File(out_h5, "w") as h5_file:
        
        for chrom, depth_array in depth_dict.items():
            chrom_group = h5_file.require_group(chrom)
            chrom_group.create_dataset("depth", data=depth_array, compression="gzip")

        h5_file.attrs["id"] = "depth"

    print(f"Finished writing in {timeit.default_timer() - start_write} seconds!")
    print(f"Frag-Depths written to {out_h5}")